In [0]:
from google.colab import drive
drive.mount('/content/drive')

import os
from google.colab import drive
drive.mount('/content/gdrive')
!pwd
os.chdir('gdrive/My Drive/IT-GCN/')
!pwd
!ls

### Re-implement graph.py to construct ajacency matrix A and normalized laplacian matrix L_hat (Graph.py)

In [0]:
### re-implement graph.py to construct ajacency matrix A and normalized laplacian matrix L_hat

import sklearn.metrics
import sklearn.neighbors
import scipy
import numpy as np
from scipy.sparse import rand


### construct ajacency matrix
def compute_dist(graph,k,metric):
  dist = scipy.spatial.distance.pdist(graph,metric)
  dist_square = scipy.spatial.distance.squareform(dist)
  id_dist = np.argsort(dist_square)[:,:k+1]
  dist_square.sort()
  dist_square = dist_square[:,:k+1]
  return dist_square,id_dist


def build_ajacency(dist,idx):
  M,k = dist.shape
  sigma = np.mean(dist[:,-1])**2
  dist = np.exp(-dist/sigma)

  # construct sparse matrix
  I = np.arange(0,M).repeat(k)
  J = idx.reshape(M*k)
  V = dist.reshape(M*k)
  W = scipy.sparse.coo_matrix((V,(I,J)), shape = (M,M))
  W.setdiag(0)
  # construct symmetric matrix
  bigger_index = W.T>W
  # W = W - W.multiply(bigger_index) + W.T.multiply(bigger_index)
  W = W + W.T.multiply(bigger_index)
  assert(type(W)) is scipy.sparse.csr.csr_matrix
  
  return W

def build_laplacian(W, normalized = True):
  d = W.sum(axis = 0)
  if not normalized:
    D = scipy.sparse.diags(d.A.squeeze(), 0)
    L = D - W
  else:   
    d += np.spacing(np.array(0,W.dtype))
    d = 1/np.sqrt(d)
    D = scipy.sparse.diags(d.A.squeeze(),0)
    I = scipy.sparse.identity(d.size,dtype= W.dtype)
    L = I - D*W*D
    
  return L

def rescale_L(L, l_max):
  L = 2/l_max * L
  I = scipy.sparse.identity(L.shape[0], format='csr', dtype=L.dtype)
  L_hat = L - I
  return L_hat
  

### Re-implement model.py to Build GCN models (Model.py)

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
import sklearn
import scipy.sparse
import numpy as np
import os, time, collections, shutil
# import graph

#NFEATURES = 28**2
#NCLASSES = 10

# Common methods for all models
class base_model(object):

    def __init__(self):
        self.regularizers = []

    # High-level interface which runs the constructed computational graph.

    # def output_feature(self, data, labels = None, sess = None):
    #     sess = self._get_session(sess)
    #     size = data.shape[0]
    #     begin = 0
    #     end = size
    #     batch_data = np.zeros((self.batch_size, self.n_views, data.shape[2], data.shape[3]))
    #     tmp_data = data[begin:end, :, :, :]
    #     if type(tmp_data) is not np.ndarray:
    #         tmp_data = tmp_data.toarray()  # convert sparse matrices
        
    #     batch_data = tmp_data
    #     feed_dict2 = {self.ph_data2: batch_data, self.ph_dropout2: 1}
    #     output_x = sess.run(self.feature_all, feed_dict2)
    #     output_x = np.array(output_x)
    #     print(output_x.shape)


    def predict(self, data, labels=None, sess=None):
        loss = 0
        size = data.shape[0]
        N,V,M,F = data.shape
        feature_size = np.array(self.F[0])
        print(feature_size, self.F)
        output_feature2 = np.zeros([N,V,M*feature_size])

        predictions = np.empty(size)
        sess = self._get_session(sess)
        output_feature = []

        for begin in range(0, size, self.batch_size):
            end = begin + self.batch_size
            end = min([end, size])
            batch_data = np.zeros((self.batch_size, self.n_views, data.shape[2], data.shape[3]))
            tmp_data = data[begin:end, :, :, :]

            if type(tmp_data) is not np.ndarray:
                tmp_data = tmp_data.toarray()  # convert sparse matrices
            batch_data[:end-begin] = tmp_data
            feed_dict = {self.ph_data: batch_data, self.ph_dropout: 1}
            # output_x = sess.run(self.feature_all, feed_dict)
            # output_feature2[begin:end,:,:] = output_x[0:end-begin,:,:]

            # Compute loss if labels are given.
            if labels is not None:
                batch_labels = np.zeros(self.batch_size)
                batch_labels[:end-begin] = labels[begin:end]
                feed_dict[self.ph_labels] = batch_labels
                batch_pred, batch_loss = sess.run([self.op_prediction, self.op_loss], feed_dict)
                loss += batch_loss
            else:
                batch_pred = sess.run(self.op_prediction, feed_dict)

            predictions[begin:end] = batch_pred[:end-begin]
        # print(output_feature2.shape)

        if labels is not None:
            return predictions, loss * self.batch_size / size, output_feature2
        else:
            return predictions


    def evaluate(self, data, labels, sess=None):
        """
        Runs one evaluation against the full epoch of data.
        Return the precision and the number of correct predictions.
        Batch evaluation saves memory and enables this to run on smaller GPUs.

        sess: the session in which the model has been trained.
        op: the Tensor that returns the number of correct predictions.
        data: size N x M
            N: number of signals (samples)
            M: number of vertices (features)
        labels: size N
            N: number of signals (samples)
        """
        t_process, t_wall = time.process_time(), time.time()
        predictions, loss, output_feature_all = self.predict(data, labels, sess)

        error = predictions-labels
        error = np.abs(error)
        error_sum = np.sum(error)
        accuracy = 1 - error_sum/error.size
        fpr, tpr, _ = sklearn.metrics.roc_curve(labels, predictions)
        auc = 100 * sklearn.metrics.auc(fpr, tpr)
        string = 'auc: {:.2f}, loss: {:.2e}'.format(auc, loss)
        print('accuracy = ', accuracy)
        if sess is None:
            string += '\ntime: {:.0f}s (wall {:.0f}s)'.format(time.process_time()-t_process, time.time()-t_wall)
        # return string, accuracy, auc, loss, predictions
        return string, auc, loss, predictions, accuracy, output_feature_all


    def fit(self, data, train_labels, val_data, val_labels):
        t_process, t_wall = time.process_time(), time.time()
        sess = tf.Session(graph=self.graph)
        shutil.rmtree(self._get_path('summaries'), ignore_errors=True)
        writer = tf.summary.FileWriter(self._get_path('summaries'), self.graph)
        shutil.rmtree(self._get_path('checkpoints'), ignore_errors=True)
        os.makedirs(self._get_path('checkpoints'))
        path = os.path.join(self._get_path('checkpoints'), 'model')
        sess.run(self.op_init)

        # Training.
        count = 0
        bad_counter = 0
        accuracies = []
        aucs = []
        losses = []
        indices = collections.deque()
        train_data_size = len(list(train_labels))
        num_steps = int(self.num_epochs * train_data_size / self.batch_size)
        estop = False  # early stop
        n, v, m, f = data.shape
        for step in range(1, num_steps+1):

            # Be sure to have used all the samples before using one a second time.
            if len(indices) < self.batch_size:
                indices.extend(np.random.permutation(train_data_size))
                # indices.extend(np.array(range(train_data_size)))
            idx = [indices.popleft() for i in range(self.batch_size)]
            count += len(idx)

            train_data = data[idx, :, :, :]
            batch_data, batch_labels = train_data, train_labels[idx]
            if type(batch_data) is not np.ndarray:
                batch_data = batch_data.toarray()  # convert sparse matrices
            feed_dict = {self.ph_data: batch_data, self.ph_labels: batch_labels, self.ph_dropout: self.dropout}
            learning_rate, loss_average = sess.run([self.op_train, self.op_loss_average], feed_dict)

            # Periodical evaluation of the model.
            if step % self.eval_frequency == 0 or step == num_steps:
                # print ('Seen samples: %d' % count)
                epoch = step * self.batch_size / train_data_size
                # print('step {} / {} (epoch {:.2f} / {}):'.format(step, num_steps, epoch, self.num_epochs))
                # print('  learning_rate = {:.2e}, loss_average = {:.2e}'.format(learning_rate, loss_average))
                # string, auc, loss, predictions, accuracy = self.evaluate(val_data, val_labels, sess)
                # aucs.append(auc)
                # losses.append(loss)
                # print('  validation {}'.format(string))
                # print(predictions.tolist()[:50])
                # print('  time: {:.0f}s (wall {:.0f}s)'.format(time.process_time()-t_process, time.time()-t_wall))

                ## Summaries for TensorBoard.
                summary = tf.Summary()
                writer.add_summary(summary, step)

                # Save model parameters (for evaluation).
                self.op_saver.save(sess, path, global_step=step)

                # if len(aucs) > (self.patience+5) and auc > np.array(aucs).max():
                #     bad_counter = 0

                # if len(aucs) > (self.patience+5) and auc <= np.array(aucs)[:-self.patience].max():
                #     bad_counter += 1
                #     if bad_counter > self.patience:
                #         print('Early Stop!')
                #         estop = True
                #         break

            if estop:
                break
        sess.close()

        t_step = (time.time() - t_wall) / num_steps
        return aucs, losses, t_step

    def get_var(self, name):
        sess = self._get_session()
        var = self.graph.get_tensor_by_name(name + ':0')
        val = sess.run(var)
        sess.close()
        return val

    # Methods to construct the computational graph.
    def build_multi_gcn_graph(self, M_0):
        """Build the computational graph of the model."""
        self.graph = tf.Graph()
        with self.graph.as_default():

            # Inputs.
            with tf.name_scope('inputs'):
                self.ph_data = tf.placeholder(tf.float32,(self.batch_size, self.n_views, M_0, self.fin), 'data')
                # self.ph_data = tf.placeholder(tf.float32,(None, self.n_views, M_0, self.fin), 'data')
                self.ph_labels = tf.placeholder(tf.int32, (self.batch_size), 'labels')
                self.ph_dropout = tf.placeholder(tf.float32, (), 'dropout')
            # Model.
            op_logits, feature_all = self.inference(self.ph_data, self.ph_dropout)

            self.my_M0 = M_0
            self.feature_all = feature_all

            self.op_loss, self.op_loss_average = self.loss(op_logits, self.ph_labels, self.regularization)
            self.op_train = self.training(self.op_loss, self.learning_rate,
                    self.decay_steps, self.decay_rate, self.momentum)
            self.op_prediction = self.prediction(op_logits)


            # Initialize variables, i.e. weights and biases.
            self.op_init = tf.global_variables_initializer()

            # Summaries for TensorBoard and Save for model parameters.
            self.op_summary = tf.summary.merge_all()
            self.op_saver = tf.train.Saver(max_to_keep=5)

        self.graph.finalize()

    # Methods to construct the computational graph.
    def build_gcn_graph(self, M_0):
        """Build the computational graph of the model."""
        self.graph = tf.Graph()
        with self.graph.as_default():

            # Inputs.
            with tf.name_scope('inputs'):
                self.ph_data = tf.placeholder(tf.float32, (self.batch_size, M_0, self.fin), 'data')
                self.ph_labels = tf.placeholder(tf.int32, (self.batch_size), 'labels')
                self.ph_dropout = tf.placeholder(tf.float32, (), 'dropout')

            # Model.
            op_logits = self.inference(self.ph_data, self.ph_dropout)
            self.op_loss, self.op_loss_average = self.loss(op_logits, self.ph_labels, self.regularization)
            self.op_train = self.training(self.op_loss, self.learning_rate,
                    self.decay_steps, self.decay_rate, self.momentum)
            self.op_prediction = self.prediction(op_logits)

            # Initialize variables, i.e. weights and biases.
            self.op_init = tf.global_variables_initializer()

            # Summaries for TensorBoard and Save for model parameters.
            self.op_summary = tf.summary.merge_all()
            self.op_saver = tf.train.Saver(max_to_keep=5)

        self.graph.finalize()


    def inference(self, data, dropout):
        """
        It builds the model, i.e. the computational graph, as far as
        is required for running the network forward to make predictions,
        i.e. return logits given raw data.

        data: size N x M
            N: number of signals (samples)
            M: number of vertices (features)
        training: we may want to discriminate the two, e.g. for dropout.
            True: the model is built for training.
            False: the model is built for evaluation.
        """
        # TODO: optimizations for sparse data
        logits, feature_all = self._inference(data, dropout)
        return logits, feature_all


    def probabilities(self, logits):
        """Return the probability of a sample to belong to each class."""
        with tf.name_scope('probabilities'):
            probabilities = tf.nn.softmax(logits)
            return probabilities

    def prediction(self, logits):
        """Return the predicted classes."""
        with tf.name_scope('prediction'):
            prediction = tf.argmax(logits, axis=1)
            # prediction = logits
            return prediction

    def loss(self, logits, labels, regularization):
        """Adds to the inference model the layers required to generate loss."""
        with tf.name_scope('loss'):
            with tf.name_scope('cross_entropy'):
                labels = tf.to_int64(labels)
                cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
                cross_entropy = tf.reduce_mean(cross_entropy)
            with tf.name_scope('regularization'):
                regularization *= tf.add_n(self.regularizers)
            loss = cross_entropy + regularization

            # Summaries for TensorBoard.
            tf.summary.scalar('loss/cross_entropy', cross_entropy)
            tf.summary.scalar('loss/regularization', regularization)
            tf.summary.scalar('loss/total', loss)
            with tf.name_scope('averages'):
                averages = tf.train.ExponentialMovingAverage(0.9)
                op_averages = averages.apply([cross_entropy, regularization, loss])
                tf.summary.scalar('loss/avg/cross_entropy', averages.average(cross_entropy))
                tf.summary.scalar('loss/avg/regularization', averages.average(regularization))
                tf.summary.scalar('loss/avg/total', averages.average(loss))
                with tf.control_dependencies([op_averages]):
                    loss_average = tf.identity(averages.average(loss), name='control')
            return loss, loss_average


    def training(self, loss, learning_rate, decay_steps, decay_rate=0.95, momentum=0.9):
        """Adds to the loss model the Ops required to generate and apply gradients."""
        with tf.name_scope('training'):
            # Learning rate.
            global_step = tf.Variable(0, name='global_step', trainable=False)
            if decay_rate != 1:
                learning_rate = tf.train.exponential_decay(
                        learning_rate, global_step, decay_steps, decay_rate, staircase=True)
            tf.summary.scalar('learning_rate', learning_rate)
            # Optimizer.
            if momentum == 0:
                optimizer = tf.train.GradientDescentOptimizer(learning_rate)
                #optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
            else:
                optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
            grads = optimizer.compute_gradients(loss)
            op_gradients = optimizer.apply_gradients(grads, global_step=global_step)
            # Histograms.
            for grad, var in grads:
                if grad is None:
                    print('warning: {} has no gradient'.format(var.op.name))
                else:
                    tf.summary.histogram(var.op.name + '/gradients', grad)
            # The op return the learning rate.
            with tf.control_dependencies([op_gradients]):
                op_train = tf.identity(learning_rate, name='control')
            return op_train

    # Helper methods.

    def _get_path(self, folder):
        # path = '../../models/'
        path = 'models/'
        return os.path.join(path, folder, self.dir_name)


    def _get_session(self, sess=None):
        """Restore parameters if no session given."""
        if sess is None:
            sess = tf.Session(graph=self.graph)
            filename = tf.train.latest_checkpoint(self._get_path('checkpoints'))
            self.op_saver.restore(sess, filename)
        return sess

    def _weight_variable(self, shape, regularization=True):
        initial = tf.truncated_normal_initializer(0, 0.1)
        var = tf.get_variable('weights', shape, tf.float32, initializer=initial)
        if regularization:
          self.regularizers.append(tf.nn.l2_loss(var))
        tf.summary.histogram(var.op.name, var)
        return var

    def _bias_variable(self, shape, regularization=True):
        initial = tf.constant_initializer(0.1)
        var = tf.get_variable('bias', shape, tf.float32, initializer=initial)
        if regularization:
            self.regularizers.append(tf.nn.l2_loss(var))
        tf.summary.histogram(var.op.name, var)
        return var

    def _conv2d(self, x, W):
        return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


class siamese_m_cgcnn(base_model):
    """
    Graph CNN which uses the Chebyshev approximation.

    The following are hyper-parameters of graph convolutional layers.
    They are lists, which length is equal to the number of gconv layers.
        F: Number of features.
        K: List of polynomial orders, i.e. filter sizes or number of hopes.
        p: Pooling size.
           Should be 1 (no pooling) or a power of 2 (reduction by 2 at each coarser level).
           Beware to have coarsened enough.

    L: List of Graph Laplacians. Size M x M. One per coarsening level.

    The following are hyper-parameters of fully connected layers.
    They are lists, which length is equal to the number of fc layers.
        M: Number of features per sample, i.e. number of hidden neurons.
           The last layer is the softmax, i.e. M[-1] is the number of classes.

    The following are choices of implementation for various blocks.
        filter: filtering operation, e.g. chebyshev5, lanczos2 etc.
        brelu: bias and relu, e.g. b1relu or b2relu.
        pool: pooling, e.g. mpool1.

    Training parameters:
        num_epochs:    Number of training epochs.
        learning_rate: Initial learning rate.
        decay_rate:    Base of exponential decay. No decay with 1.
        decay_steps:   Number of steps after which the learning rate decays.
        momentum:      Momentum. 0 indicates no momentum.

    Regularization parameters:
        regularization: L2 regularizations of weights and biases.
        dropout:        Dropout (fc layers): probability to keep hidden neurons. No dropout with 1.
        batch_size:     Batch size. Must divide evenly into the dataset sizes.
        eval_frequency: Number of steps between evaluations.

    Directories:
        dir_name: Name for directories (summaries and model parameters).
    """
    def __init__(self, L, F, K, p, M, fin, n_views, view_com, method='gcn', filter='chebyshev5', brelu='b1relu', pool='mpool1',
                num_epochs=20, learning_rate=0.01, decay_rate=0.95, decay_steps=None, momentum=0.9,
                regularization=0, dropout=0, batch_size=100, eval_frequency=200, patience=10,
                dir_name=''):
        super().__init__()

        # Verify the consistency w.r.t. the number of layers.
        assert len(L) >= len(F) == len(K) == len(p)
        assert np.all(np.array(p) >= 1)
        p_log2 = np.where(np.array(p) > 1, np.log2(p), 0)
        assert np.all(np.mod(p_log2, 1) == 0)  # Powers of 2.
        assert len(L) >= 1 + np.sum(p_log2)  # Enough coarsening levels for pool sizes.

        # Keep the useful Laplacians only. May be zero.
        M_0 = L[0].shape[0]
        j = 0
        self.L = []
        for pp in p:
            self.L.append(L[j])
            j += int(np.log2(pp)) if pp > 1 else 0
        L = self.L

        # Print information about NN architecture.
        Ngconv = len(p)
        Nfc = len(M)
        # print('NN architecture')
        # print('  input: M_0 = {}'.format(M_0))
        for i in range(Ngconv):
            # print('  layer {0}: cgconv{0}'.format(i+1))
            # print('    representation: M_{0} * F_{1} / p_{1} = {2} * {3} / {4} = {5}'.format(
            #         i, i+1, L[i].shape[0], F[i], p[i], L[i].shape[0]*F[i]*fin//p[i]))
            F_last = F[i-1] if i > 0 else 1
            # print('    weights: F_{0} * F_{1} * K_{1} = {2} * {3} * {4} = {5}'.format(
            #         i, i+1, F_last, F[i], K[i], F_last*F[i]*K[i]))
            # if brelu == 'b1relu':
                # print('    biases: F_{} = {}'.format(i+1, F[i]))
            # elif brelu == 'b2relu':
                # print('    biases: M_{0} * F_{0} = {1} * {2} = {3}'.format(
                #         i+1, L[i].shape[0], F[i], L[i].shape[0]*F[i]))
        for i in range(Nfc):
            name = 'logits (softmax)' if i == Nfc-1 else 'fc{}'.format(i+1)
            # print('  layer {}: {}'.format(Ngconv+i+1, name))
            # print('    representation: M_{} = {}'.format(Ngconv+i+1, M[i]))
            M_last = M[i-1] if i > 0 else M_0 if Ngconv == 0 else L[-1].shape[0] * F[-1] // p[-1]
            # print('    weights: M_{} * M_{} = {} * {} = {}'.format(
            #         Ngconv+i, Ngconv+i+1, M_last, M[i], M_last*M[i]))
            # print('    biases: M_{} = {}'.format(Ngconv+i+1, M[i]))

        # Store attributes and bind operations.
        self.n_views, self.view_com = n_views, view_com
        self.data_size = 7
        self.L, self.F, self.K, self.p, self.M, self.fin = L, F, K, p, M, fin
        self.num_epochs, self.learning_rate, self.patience = num_epochs, learning_rate, patience
        self.decay_rate, self.decay_steps, self.momentum = decay_rate, decay_steps, momentum
        self.regularization, self.dropout = regularization, dropout
        self.batch_size, self.eval_frequency = batch_size, eval_frequency
        self.dir_name = dir_name
        self.method = method
        self.filter = getattr(self, filter)
        self.brelu = getattr(self, brelu)
        self.pool = getattr(self, pool)
        # Build the computational graph.
        self.build_multi_gcn_graph(M_0)


    def chebyshev5(self, x, L, Fout, K):
        if(self.batch_size == 1):
          x = x[np.newaxis]
        N, M, Fin = x.get_shape()
        N, M, Fin = int(N), int(M), int(Fin)
        # Rescale Laplacian and store as a TF sparse tensor. Copy to not modify the shared L.
        L = scipy.sparse.csr_matrix(L)
        L = rescale_L(L, 2)
        L = L.tocoo()
        indices = np.column_stack((L.row, L.col))
        L = tf.SparseTensor(indices, L.data, L.shape)
        L = tf.sparse_reorder(L)
        # Transform to Chebyshev basis
        x0 = tf.transpose(x, perm=[1, 2, 0])  # M x Fin x N
        x0 = tf.reshape(x0, [M, Fin*N])  # M x Fin*N
        x = tf.expand_dims(x0, 0)  # 1 x M x Fin*N
        def concat(x, x_):
            x_ = tf.expand_dims(x_, 0)  # 1 x M x Fin*N
            return tf.concat([x, x_], axis=0)  # K x M x Fin*N
        if K > 1:
            x1 = tf.sparse_tensor_dense_matmul(L, x0)
            x = concat(x, x1)
        for k in range(2, K):
            x2 = 2 * tf.sparse_tensor_dense_matmul(L, x1) - x0  # M x Fin*N
            x = concat(x, x2)
            x0, x1 = x1, x2
        x = tf.reshape(x, [K, M, Fin, N])  # K x M x Fin x N
        x = tf.transpose(x, perm=[3,1,2,0])  # N x M x Fin x K
        x = tf.reshape(x, [N*M, Fin*K])  # N*M x Fin*K
        # Filter: Fin*Fout filters of order K, i.e. one filterbank per feature pair.
        W = self._weight_variable([Fin*K, Fout], regularization=False)
        x = tf.matmul(x, W)  # N*M x Fout
        return tf.reshape(x, [N, M, Fout])  # N x M x Fout

    def b1relu(self, x):
        """Bias and ReLU. One bias per filter."""
        N, M, F = x.get_shape()
        b = self._bias_variable([1, 1, int(F)], regularization=False)
        return tf.nn.relu(x + b)

    def b2relu(self, x):
        """Bias and ReLU. One bias per vertex per filter."""
        N, M, F = x.get_shape()
        b = self._bias_variable([1, int(M), int(F)], regularization=False)
        return tf.nn.relu(x + b)

    def mpool1(self, x, p):
        """Max pooling of size p. Should be a power of 2."""
        if p > 1:
            x = tf.expand_dims(x, 3)  # N x M x F x 1
            x = tf.nn.max_pool(x, ksize=[1,p,1,1], strides=[1,p,1,1], padding='SAME')
            #tf.maximum
            return tf.squeeze(x, [3])  # N x M/p x F
        else:
            return x

    def apool1(self, x, p):
        """Average pooling of size p. Should be a power of 2."""
        if p > 1:
            x = tf.expand_dims(x, 3)  # N x M x F x 1
            x = tf.nn.avg_pool(x, ksize=[1,p,1,1], strides=[1,p,1,1], padding='SAME')
            return tf.squeeze(x, [3])  # N x M/p x F
        else:
            return x

    def fc(self, x, Mout, relu=False):
        """Fully connected layer with Mout features."""
        N, Min = x.get_shape()
        W = self._weight_variable([int(Min), Mout], regularization=True)
        b = self._bias_variable([Mout], regularization=True)
        x = tf.matmul(x, W) + b
        return tf.nn.relu(x) if relu else x
        
        
    def _inference_single(self, x, dropout, name, reuse=False):
        x_0 = tf.squeeze(x)

        with tf.variable_scope("siamese", reuse=reuse) as scope:
            for i in range(len(self.p)):
                with tf.variable_scope('conv1{}'.format(i+1)):
                    with tf.name_scope('filter'):
                        x_0 = self.filter(x_0, self.L[i], self.F[i], self.K[i])
                    with tf.name_scope('bias_relu'):
                        x_0 = self.brelu(x_0)
                    with tf.name_scope('pooling'):
                        x_0 = self.pool(x_0, self.p[i])

        return x_0

    def _view_pool(self, view_features, name, method='max'):
        """Max pooling of size p. Should be a power of 2."""

        vp = tf.expand_dims(view_features[0], 0) # eg. [100] -> [1, 100]
        for v in view_features[1:]:
            v = tf.expand_dims(v, 0)
            vp = tf.concat([vp, v], axis=0)
        # print ('vp before reducing:', vp.get_shape().as_list())
        if method == 'max':
            vp = tf.reduce_max(vp, [0], name=name)
        elif method == 'mean':
            vp = tf.reduce_mean(vp, [0], name=name)
        return vp   

    def _inference(self, views, dropout):
        """views: N x V x M x F tensor"""

        n_views = views.get_shape().as_list()[1]
        _,V,_,_ = views.get_shape()
        # transpose views : (NxVxMxF) -> (VxNxMxF)
        views = tf.transpose(views, perm=[1, 0, 2, 3])

        view_pool_0 = []
        for i in range(n_views):

            # set reuse True for i > 0, for weight-sharing
            reuse = (i != 0)
            # reuse = False
            view = tf.gather(views, i) # NxMxF

            x_0 = self._inference_single(view, dropout, i, reuse)
            # x_0 = tf.nn.l2_normalize(x_0, dim=2, epsilon=1e-12, name=None)
            N, M, F = x_0.get_shape()
            x_0 = tf.reshape(x_0, [int(N), int(M * F)])
            view_pool_0.append(x_0)

        # view_pool_0 = tf.reshape(view_pool_0,[int(N), int(V)])
        feature_all = tf.reshape(view_pool_0,[int(N), int(V), int(M * F)])
        # max pooling for views
        pool_vp_0 = self._view_pool(view_pool_0, 'pool_vp', self.view_com)
        # print ('pool_vp_0', pool_vp_0.get_shape().as_list())

        # Dot product layer
        x_0 = tf.reshape(pool_vp_0, [int(N * M), int(F)])
        x_0 = tf.nn.l2_normalize(x_0, dim=1, epsilon=1e-12, name=None)

        # x_ = tf.reduce_sum(x_, 1, keep_dims=True)
        x_ = tf.reshape(x_0, [int(N), int(M), int(F)])
    
        # Fully connected hidden layers.
        N, M, F = x_.get_shape()
        # print(N,M,F)
        x_ = tf.reshape(x_, [int(N), int(M*F)])  # N x M x F
        x1 = x_
        for i, M in enumerate(self.M[:-1]):
            with tf.variable_scope('fc{}'.format(i+1)):
                x_ = self.fc(x_, M)
                x_ = tf.nn.dropout(x_, dropout)

        # Logits linear layer, i.e. softmax without normalization.
        with tf.variable_scope('logits'):
            x_ = self.fc(x_, self.M[-1], relu=False)

        return x_, feature_all  

TensorFlow 1.x selected.


### Define Training (Train.py)

In [0]:
# %load_ext autoreload
# %autoreload 2
from __future__ import division
from __future__ import print_function

import sys, os
sys.path.insert(0, '..')
# import models, graph, coarsening, utils
# from utils import model_perf

import tensorflow as tf
import numpy as np
import time
import argparse

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

import pandas as pd
import scipy.sparse as sp
import pickle as pkl
from sklearn.metrics import confusion_matrix


# %matplotlib inline
flags = tf.app.flags
FLAGS = flags.FLAGS

# neural network setting
# Graphs.
# flags.DEFINE_integer('number_edges', 8, 'Graph: minimum number of edges per vertex.')
# flags.DEFINE_string('metric', 'euclidean', 'Graph: similarity measure (between features).')
# TODO: change cgcnn for combinatorial Laplacians.
# flags.DEFINE_bool('normalized_laplacian', True, 'Graph Laplacian: normalized.')
# flags.DEFINE_integer('coarsening_levels', 4, 'Number of coarsened graphs.')

results_auc = dict()
results = list()

class model_perf(object):

    def __init__(self):
        self.names, self.params = set(), {}
        self.fit_auc, self.fit_losses, self.fit_time = {}, {}, {}
        self.train_auc, self.train_loss = {}, {}
        self.test_auc, self.test_loss = {}, {}
        self.train_represent = {}


    def test(self, model, name, params, data, train_labels, val_data, val_labels, test_chosen_index, test_labels, mode, label_all):
        self.params[name] = params
        self.fit_auc[name], self.fit_losses[name], self.fit_time[name] = model.fit(data, train_labels, val_data, val_labels)

        # del val_data, val_labels
        n, v, m, f = data.shape
        if params['method'] == 'gcn' or params['method'] == '2gcn':
            test_data = data[test_chosen_index[:], :, :, :]
        elif params['method'] == 'fnn' or params['method'] == '2fnn':
            new_data = np.zeros([n, v, m*f])
            for i in range(n):
                for j in range(v):
                    new_data[i, j, :] = data[i, j, :, :].flatten()
            test_data = new_data[test_chosen_index[:], :, :]

        print('****** start_testing *******')
        if(mode == 'validate'):
          # string, self.test_auc[name], self.test_loss[name], predictions, accuracy,_ = model.evaluate(test_data, test_labels)
          string, self.test_auc[name], self.test_loss[name], predictions, accuracy,_ = model.evaluate(val_data, val_labels)
          # print(val_data.shape)
        elif(mode == 'test'):
          string, self.test_auc[name], self.test_loss[name], predictions, accuracy,_ = model.evaluate(test_data, test_labels)
          print(predictions)
          # string, self.test_auc[name], self.test_loss[name], predictions, accuracy, output_feature_all = model.evaluate(data, label_all)
          # N1,V1,F1 = output_feature_all.shape
          # output_feature_reshape = np.reshape(output_feature_all,[int(N1),int(V1*F1)])
          # print('now we write')
          # np.savetxt(r'results/HIV_feature_all_input_HOSVD_m_30.txt',output_feature_reshape,fmt="%f")
        # model.output_feature(test_data, test_labels)
        # print('test  {}'.format(string))

        # if(mode == 'test'):
        #   test_labels = label_all

        fpr, tpr, _ = sklearn.metrics.roc_curve(test_labels, predictions)
        auc = 100 * sklearn.metrics.auc(fpr, tpr)
        tn, fp, fn, tp = confusion_matrix(test_labels, predictions).ravel()
        sensitivity = tp/(tp+fn)
        specificity = tn/(tn+fp) 


        self.names.add(name)
        return accuracy, auc, sensitivity, specificity
        

    def fin_result(self, data_type, i_fold=None):
        for name in sorted(self.names):
            if name not in results_auc:
                results_auc[name] = 0
            results_auc[name] += self.test_auc[name]
            results.append([i_fold, self.test_auc[name]])
        if i_fold == 4:
            for name in sorted(self.names):
                results_auc[name] /= 5
                print('{:5.2f}  {}'.format(
                    results_auc[name], name))
            results.append([name, results_auc[name]])
            # r = pd.DataFrame(data=results)
            # r.to_csv('../../../data/output/' + data_type + '_fin_results', index=False, header=['method', 'test_auc'])


    def show(self, fontsize=None):
        if fontsize:
            plt.rc('pdf', fonttype=42)
            plt.rc('ps', fonttype=42)
            plt.rc('font', size=fontsize)         # controls default text sizes
            plt.rc('axes', titlesize=fontsize)    # fontsize of the axes title
            plt.rc('axes', labelsize=fontsize)    # fontsize of the x any y labels
            plt.rc('xtick', labelsize=fontsize)   # fontsize of the tick labels
            plt.rc('ytick', labelsize=fontsize)   # fontsize of the tick labels
            plt.rc('legend', fontsize=fontsize)   # legend fontsize
            plt.rc('figure', titlesize=fontsize)  # size of the figure title
        # print('  auc      loss        time [ms]  name')
        # print('test  train   test  train   test     train')
        # for name in sorted(self.names):
        #     print('{:5.2f} {:5.2f}   {:.2e} {:.2e}   {:3.0f}   {}'.format(
        #             self.test_auc[name], self.train_auc[name],
        #             self.test_loss[name], self.train_loss[name], self.fit_time[name]*1000, name))



def get_feed_data(data, subj, indexes, labels, method='gcn'):
    train_indexes, val_indexes, test_indexes = indexes
    train_labels, val_labels, test_labels = labels
    n, v, m, f = data.shape
    # if v == 6:
    #     print (val_labels.shape)
    #     n_val_indexes = 10000
    #     sidx = np.random.permutation(val_labels.shape[0])
    #     val_indexes =  np.array([val_indexes[s] for s in sidx[:n_val_indexes]])
    #     val_labels = np.array([val_labels[s] for s in sidx[:n_val_indexes]])
    # f = 1 # whether f can be deleted
    if method == 'gcn' or method == '2gcn':
        val_x = data[val_indexes[:], :, :, :]
    elif method == 'fnn' or method == '2fnn':
        new_data = np.zeros([n, v, m*f])
        for i in range(n):
            for j in range(v):
                new_data[i, j, :] = data[i, j, :, :].flatten()
        val_x = new_data[val_indexes[:], :, :]

    train_y = train_labels
    val_y = val_labels
    test_y = test_labels
    del subj
    del train_labels, val_labels, test_labels
    del val_indexes
    return train_indexes, train_y, val_x, val_y, test_indexes, test_y


def train(method, view_com, n_views, k, theta, m, n_epoch, batch_size, pairs, labels, node_features, subj, data, data_type, i_fold, mode, label_all):
    str_params = view_com + '_k' + str(k) + '_m' + str(m) + '_'
    obj_params = 'softmax'

    # print ('Construct ROI graphs...')
    t_start = time.process_time()

    dist, idx = compute_dist(node_features, k, metric = 'euclidean')
    A = build_ajacency(dist, idx).astype(np.float32)

    if method == '2gcn':
        graphs, perm = coarsen(A, levels=FLAGS.coarsening_levels, self_connections=False)
        # graphs, perm = coarsen(A, levels = 6, self_connections=False)
        L = [build_laplacian(A, normalized=True) for A in graphs]
        data = perm_data1(data, perm, True)
    else:
        graphs = list()
        graphs.append(A)
        L = [build_laplacian(A, normalized=True)]
        

    # print('Execution time: {:.2f}s'.format(time.process_time() - t_start))
    # graph.plot_spectrum(L)
    del A

    print ('Set parameters...')
    mp = model_perf()
    # Architecture.
    common = {}
    # common['dir_name']       = 'bp/'
    common['dir_name']       = 'HIV/'
    common['num_epochs']     = n_epoch
    common['batch_size']     = batch_size
    common['eval_frequency'] = 5 * common['num_epochs']
    common['patience']       = 5
    common['regularization'] = 5e-3
    common['dropout']        = 0
    common['learning_rate']  = 1e-2
    common['decay_rate']     = 0.95
    common['momentum']       = 0.9
    common['n_views']        = n_views
    common['view_com']       = view_com

    print ('Get feed indexes and labels...')
    train_indexes, train_y, val_x, val_y, test_indexes, test_y = get_feed_data(data, subj, pairs, labels, method)
    # print(train_indexes)
    # print(val_x)
    # print(test_indexes)

    C = max(train_y)+1
    common['decay_steps']    = train_indexes.shape[0] / (common['batch_size'] * 5)


    if method == 'gcn':
        # str_params += 'b_max_eu_'
        name = 'mvgcn'
        params = common.copy()
        params['method'] = 'gcn'
        params['F']              = [m] # filters
        params['K']              = [theta] # supports
        params['p']              = [1]
        params['M']              = [C]
        params['fin'] = val_x.shape[3]
        params['dir_name'] += name
        params['filter'] = 'chebyshev5'
        params['brelu'] = 'b2relu'
        params['pool'] = 'apool1'
        accuracy, auc, sensitivity, specificity = mp.test(siamese_m_cgcnn(L, **params), name, params,
                        data, train_y, val_x, val_y, test_indexes, test_y, mode, label_all)
        return accuracy, auc, sensitivity, specificity


    # mp.save(data_type)
    method_type = method + '_'
    # mp.fin_result(method_type + data_type + str_params + obj_params, i_fold)
    mp.fin_result(method_type + 'str' + str_params + obj_params, i_fold)

### Prepare for input parameters

In [0]:
import sklearn, sklearn.datasets
import sklearn.naive_bayes, sklearn.linear_model, sklearn.svm, sklearn.neighbors, sklearn.ensemble
import matplotlib.pyplot as plt
import scipy.sparse
import numpy as np
import time, re
import pickle as pkl
import pandas as pd
import scipy.io as sio
from  scipy import *
import random

from sklearn.model_selection import StratifiedKFold

def convert_minus_one_2_zero(label):
  for i in range(len(label)):
    if(label[i]<0):
      label[i] = 0;
  return label    

def load_data(f):
  k = 0
  input_data = {}

  for line in f:
    input_data[k] = np.int32(line.split())
    k = k + 1
  
  return input_data


def load_train_index(f):
  train_index = []
  k = 0
  for line in f:
    train_index.append(np.int32(line))

  return train_index
    


def load_data_my_new(data_name, train_fold_chosen):
  # load data from google drive
  if(data_name == 'BP'):
    x = scipy.io.loadmat('BP.mat')
    X_normalize = x['X_normalize']
    label = x['label']
  elif(data_name == 'HIV'):
    x = scipy.io.loadmat('HIV.mat') 
    X_normalize = x['X'] 
    label = x['label']
  elif(data_name == 'PPMI'):
    x = scipy.io.loadmat('PPMI.mat') 
    X_normalize = x['X'] 
    label = x['label']

  # reshape data_size to N,V,M,F  
  N_subject = X_normalize.size
  X_1 = X_normalize[0][0]
  M,F,V = X_1.shape
  print('loading data:',data_name,'of size:',N_subject,V,M,F)
  data = np.zeros([N_subject,V,M,F])
  for i in range(N_subject):
    X_i = X_normalize[i][0]
    for j in range(V):
      X_ij = X_i[:,:,j]
      data[i,j,:,:] = X_ij

  labels = np.array(label)
  if(data_name == 'BP'):
    f_test = open('results/BP_divide_test_index.txt','r')
    test_idx = load_data(f_test)
    f_train = open('results/BP_divide_train_index.txt','r')
    train_idx = load_data(f_train)
  elif(data_name == 'HIV'):
    f_test = open('results/HIV_divide_test_index.txt','r')
    test_idx = load_data(f_test)
    f_train = open('results/HIV_divide_train_index.txt','r')
    train_idx = load_data(f_train)

  f_test.close()
  f_train.close()

  train_set_ratio = 0.8
  train_chosen_idx = train_idx[train_fold_chosen]
  data_size = train_chosen_idx.size;  
  train_data_size = np.floor(data_size*train_set_ratio)
  train_index = random.sample(list(train_chosen_idx), int(train_data_size))
  test_index = np.setdiff1d(train_chosen_idx,train_index)

  labels_set = list()
  indexes_set = list()

  train_label, test_label = labels[train_index], labels[test_index]
  val_label,val_index = test_label,test_index
  train_label = convert_minus_one_2_zero(train_label)
  val_label = convert_minus_one_2_zero(val_label)
  test_label = convert_minus_one_2_zero(test_label)

  train_label = np.array(train_label).flatten()
  val_label = np.array(val_label).flatten()
  test_label = np.array(test_label).flatten()

  train_index = np.array(train_index).flatten()
  val_index = np.array(val_index).flatten()
  test_index = np.array(test_index).flatten()

  labels_set.append((train_label,val_label,test_label))
  indexes_set.append((train_index,val_index,test_index))

  train_index_all = train_chosen_idx
  test_index_all = test_idx

  subj = 0
  return data, subj, indexes_set, labels_set, train_index_all, test_index_all, convert_minus_one_2_zero(labels)


### Train CP Feature

In [0]:
!pip install git+https://github.com/ahwillia/tensortools

import tensortools as tt
import numpy as np
import time


def train_U_CP(data_name, R):
  if(data_name == 'BP'):
    x = scipy.io.loadmat('BP.mat')
    X_normalize = x['X_normalize']

  elif(data_name == 'HIV'):
    x = scipy.io.loadmat('HIV.mat') 
    X_normalize = x['X'] 

  elif(data_name == 'PPMI'):
    x = scipy.io.loadmat('PPMI.mat') 
    X_normalize = x['X'] 

  N = X_normalize.size
  X_1 = X_normalize[0][0]
  M,F,V = X_1.shape
  # print('loading data:',data_name,'of size:',N,V,M,F)
  data = np.zeros([M,F,V,N])
  for i in range(N):
    X_i = X_normalize[i][0]
    for j in range(V):
      X_ij = X_i[:,:,j]
      data[:,:,j,i] = X_ij

  data_select = data
  F = tt.cp_als(data_select, rank=R, verbose=False)

  U = F.factors[0]
  

  return U

### Train HOSVD Feature

In [0]:
!pip install tensorly
import tensorly as tl
import time
from  scipy import *


def train_U(data_name, R):
  if(data_name == 'BP'):
    x = scipy.io.loadmat('BP.mat')
    X_normalize = x['X_normalize']

  elif(data_name == 'HIV'):
    x = scipy.io.loadmat('HIV.mat') 
    X_normalize = x['X'] 

  elif(data_name == 'PPMI'):
    x = scipy.io.loadmat('PPMI.mat') 
    X_normalize = x['X'] 

  N = X_normalize.size
  X_1 = X_normalize[0][0]
  M,F,V = X_1.shape
  # print('loading data:',data_name,'of size:',N,V,M,F)
  data = np.zeros([N,M,F,V])
  for i in range(N):
    X_i = X_normalize[i][0]
    for j in range(V):
      X_ij = X_i[:,:,j]
      data[i,:,:,j] = X_ij

  data_select = data
  X1 = tl.unfold(data_select, mode=1)
  B = np.matmul(X1,X1.transpose())
  U, S, V = np.linalg.svd(B, full_matrices=True)
  S = np.sqrt(S)
  sum_all_S = np.sum(S)
  len_S = len(S)
  sum_part_S = 0
  for i in range(len_S):
    sum_part_S = sum_part_S + S[i]
    if(sum_part_S>sum_all_S*R):
        break

  U = U[:,0:i]
  return U


### Main

In [0]:
import time
localtime = time.asctime( time.localtime(time.time()) )
print ("Local time :", localtime)
print('##$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$###############$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$*****************')

method = 'gcn'
view_com = 'max'
n_views = 2
data_name = 'HIV'

if(data_name == 'HIV'):
  n_views = 2
  num_sample = 70
elif(data_name == 'BP'):
  n_views = 2
  num_sample = 97

index_set_all = np.array(range(num_sample))
R = 0.98
data_type = str
i_fold = 1
n_epoch = 30
batch_size = 1
accuracy_all = 0
accuracy_record = []
param_record = {}
iter = 0
kfold = 10

accuracy_total_sum = 0
auc_total_sum = 0
sensitivity_total_sum = 0
specificity_total_sum = 0

accuracy_total_record = []
auc_total_record = []
sensitivity_total_record = []
specificity_total_record = []

node_features = train_U_CP(data_name, 50)



for train_fold_chosen in range(kfold):
  print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
  print('start training the model of fold:',train_fold_chosen)
  print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
  data, subj, indexes_set, labels_set, train_index_all, test_index_all, label_all = load_data_my_new(data_name, train_fold_chosen)
  iter = 0
  indexes = indexes_set[0]
  labels = labels_set[0]
  accuracy_record = []
  param_record = {}
  label_all = label_all[:,0]
  mode = 'validate'


  for m in range(80,100,20):
    for k in range(8,10,2):
      for batch_size in range(2,4,2):
        theta = k
        accuracy, auc, sensitivity, specificity = train(method, view_com, n_views, k, theta, m, n_epoch, batch_size, indexes, labels, node_features, subj, data, data_type, train_fold_chosen, mode, label_all)
        print('validate_accuracy = ',accuracy)
        accuracy_record.append(accuracy)
        param_set = [k, m, n_epoch, batch_size, theta]
        param_record[iter] = param_set
        print('****************************************')

        print('The input parameter is: ','k = ',k,'m = ',m,'n_epoch = ',n_epoch, 'batch_size= ',batch_size, 'theta_size = ',theta)
        iter += 1


  max_accuracy = max(accuracy_record)
  max_index = np.argmax(np.array(accuracy_record))
  params = param_record[max_index]
  k = params[0]
  m = params[1]
  n_epoch = params[2]
  batch_size = params[3]
  theta = params[4]
  print('max_accuracy is: ', max_accuracy, 'The input parameter is: ','k = ',k,'m = ',m,'n_epoch = ',n_epoch, 'batch_size= ',batch_size, 'theta_size = ',theta)


# Retrain the model using all index
  print('start retraining the model of fold:',train_fold_chosen)
  print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
  test_index = test_index_all[train_fold_chosen]
  test_label = label_all[test_index]
  # test_label = test_label[:,0]
  print(test_label.shape)
  val_index, val_label = test_index, test_label
  train_index = train_index_all
  train_label = label_all[train_index]
  # train_label = train_label[:,0]
  print(train_label.shape)

  mode = 'test'

  labels_set = [train_label, val_label, test_label]
  indexes_set = [train_index, val_index, test_index]
  accuracy, auc, sensitivity, specificity  = train(method, view_com, n_views, k, theta, m, n_epoch, batch_size, indexes_set, labels_set, node_features, subj, data, data_type, i_fold, mode, label_all)
  print('fold: ',train_fold_chosen, 'ccuracy = ',accuracy, 'auc = ',auc, 'sensitivity = ', sensitivity, 'specificity = ',specificity)

  accuracy_total_sum = accuracy_total_sum + accuracy
  auc_total_sum = auc_total_sum + auc
  sensitivity_total_sum = sensitivity_total_sum + sensitivity
  specificity_total_sum = specificity_total_sum + specificity


  accuracy_total_record.append(accuracy)
  auc_total_record.append(auc)
  sensitivity_total_record.append(sensitivity)
  specificity_total_record.append(specificity)
  print('****************************************')

print('**********************')
average_accuracy = accuracy_total_sum/(train_fold_chosen+1)
average_auc = auc_total_sum/(train_fold_chosen+1)
average_sensitivity = sensitivity_total_sum/(train_fold_chosen+1)
average_specificity = specificity_total_sum/(train_fold_chosen+1)
/
std_value = np.std(accuracy_total_record, ddof = 1)
print('accuracy_all = ', accuracy_total_record)
print('auc_all = ', auc_total_record)
print('sensitivity_all = ', sensitivity_total_record)
print('specificity_all = ', specificity_total_record)

print('view_com = ', view_com, 'test_data', data_name, 'average_accuracy = ',average_accuracy, 'average_auc = ',average_auc, 'average_sensitivity = ',average_sensitivity, 'average_specificity= ', average_specificity, 'std_value = ', std_value)
print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')

Local time : Sun Jun 14 22:55:49 2020
##$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$###############$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$*****************
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
start training the model of fold: 0
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
loading data: HIV of size: 70 2 90 90
Set parameters...
Get feed indexes and labels...
****** start_testing *******
80 [80]
INFO:tensorflow:Restoring parameters from models/checkpoints/HIV/mvgcn/model-750
accuracy =  0.3846153846153846
validate_accuracy =  0.3846153846153846
****************************************
The input parameter is:  k =  8 m =  80 n_epoch =  30 batch_size=  2 theta_size =  8
max_accuracy is:  0.3846153846153846 The input parameter is:  k =  8 m =  80 n_epoch =  30 batch_size=  2 theta_size =  8
start retraining the model of fold: 0
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
(7,)
(63,)
Set parameters...
Get feed indexes a